In [25]:
%%sh
sudo chmod +777 utils/setup.sh
./utils/setup.sh

The user has root access.
nvidia-docker2 already installed. We are good to go!
0f22683c1dbf029997203ccf78610a2ae90162d8bb62728e3ec816ae2ebe029b
SageMaker instance routing for Docker is ok. We are good to go!


# docker单元测试

1. 修改docker sysconfig更改image存放路径
sudo vim /etc/sysconfig/docker
OPTIONS="--selinux-enabled -g /home/ec2-user/SageMaker/docker"

sudo service docker restart

2. nvidia-smi 查看cuda版本

https://www.paddlepaddle.org.cn/install/quick?docurl=/documentation/docs/zh/install/docker/linux-docker.html

nvidia-docker pull registry.baidubce.com/paddlepaddle/paddle:2.4.1-gpu-cuda11.2-cudnn8.2-trt8.0
nvidia-docker run --name paddle -p 8080:8080 -it -v $PWD:/paddle registry.baidubce.com/paddlepaddle/paddle:2.4.1-gpu-cuda11.2-cudnn8.2-trt8.0 /bin/bash 

3. 运行

mkdir /opt/program/
cd /opt/program/
pip3 install -i https://mirror.baidu.com/pypi/simple --upgrade pip
pip3 install -i https://mirror.baidu.com/pypi/simple networkx==2.3 flask gevent gunicorn boto3
pip3 install -i https://mirror.baidu.com/pypi/simple paddlenlp onnx onnxconverter_common onnxruntime-gpu nvgpu
apt-get -y update && apt-get install -y --no-install-recommends          wget          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
./serve 


In [51]:
#locally
url='http://localhost:8080/invocations'
texts = ['sagemaker-kwm-new','债务人: cksdafgsfgsjhkvgdkjsV 破产管理人:kjLFHk;fhs;dhsd;f']
payload = json.dumps(texts)
headers = {'Content-Type': 'application/json'}
r = requests.post(url,data=payload,headers=headers)
r.json()

065jt82yko-algo-1-wes65 | ================ INVOCATIONS =================
065jt82yko-algo-1-wes65 | <<<< flask.request.content_type application/json
065jt82yko-algo-1-wes65 | gpu:0
065jt82yko-algo-1-wes65 | [2023-01-17 07:52:01,977] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load '/opt/program/'.
065jt82yko-algo-1-wes65 | [2023-01-17 07:52:01,977] [    INFO] - loading configuration file /opt/program/config.json
065jt82yko-algo-1-wes65 | [2023-01-17 07:52:01,978] [    INFO] - Model config ErnieConfig {
065jt82yko-algo-1-wes65 |   "architectures": [
065jt82yko-algo-1-wes65 |     "ErnieForSequenceClassification"
065jt82yko-algo-1-wes65 |   ],
065jt82yko-algo-1-wes65 |   "attention_probs_dropout_prob": 0.1,
065jt82yko-algo-1-wes65 |   "enable_recompute": false,
065jt82yko-algo-1-wes65 |   "fuse": false,
065jt82yko-algo-1-wes65 |   "hidden_act": "gelu",
065jt82yko-algo-1-wes65 |   "hidden_dropout_prob": 0.1,
065jt82yko-algo-1-w

[0]

In [45]:
r

<Response [502]>

# sagemaker集成测试

In [50]:
%%time

!sh build_and_push.sh kwm-new

import os
import boto3
from time import gmtime, strftime
from datetime import datetime

import sagemaker
from sagemaker import get_execution_role
from sagemaker.model import Model

role = get_execution_role()

sage = boto3.Session().client(service_name='sagemaker') 

container = f'969422986683.dkr.ecr.cn-northwest-1.amazonaws.com.cn/kwm-new:latest'

sagemaker_role = sagemaker.get_execution_role()
sess = sagemaker.Session()

model = Model(image_uri=container, 
              model_data="file://file.tar.gz",
              role=sagemaker_role)

resource_name = "kwm-new-{}"
endpoint_name = resource_name.format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

initial_instance_count=1
instance_type='ml.g4dn.xlarge'

predictor = model.deploy(
    initial_instance_count=initial_instance_count,
    instance_type='local_gpu',
    endpoint_name=endpoint_name,
)

cn-northwest-1
969422986683.dkr.ecr.cn-northwest-1.amazonaws.com.cn/kwm-new:latest
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  776.3MB
Step 1/15 : FROM registry.baidubce.com/paddlepaddle/paddle:2.4.1-gpu-cuda10.2-cudnn7.6-trt7.0
 ---> 98e68d139b09
Step 2/15 : ENV LANG=en_US.utf8
 ---> Using cache
 ---> cbe993f09e08
Step 3/15 : ENV LANG=C.UTF-8
 ---> Using cache
 ---> 6da2

In [55]:
%%time

# !sh build_and_push.sh kwm-new

import os
import boto3
from time import gmtime, strftime
from datetime import datetime

import sagemaker
from sagemaker import get_execution_role
from sagemaker.model import Model

role = get_execution_role()

sage = boto3.Session().client(service_name='sagemaker') 

container = f'969422986683.dkr.ecr.cn-northwest-1.amazonaws.com.cn/kwm-new:latest'

sagemaker_role = sagemaker.get_execution_role()
sess = sagemaker.Session()

model = Model(image_uri=container, 
              role=sagemaker_role)

resource_name = "kwm-new-{}"
endpoint_name = resource_name.format(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))

initial_instance_count=1
instance_type='ml.g4dn.xlarge'

predictor = model.deploy(
    initial_instance_count=initial_instance_count,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
)

-----------------!CPU times: user 638 ms, sys: 28.4 ms, total: 667 ms
Wall time: 8min 32s


In [56]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import sagemaker
endpoint_name = 'kwm-new-2023-01-17-07-59-23'  
predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name)

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

texts = ['sagemaker-kwm-new','被告吉林紫鑫药业股份有限公司于本判决生效后十日内给付原告江苏启安建设集团有限公司工程款1，266，485.20元']

import time
start = time.time()
outputs = predictor.predict(texts)
end = time.time()
print('outputs: ', outputs)
print('time:', end-start)


outputs:  [0]
time: 11.58616590499878


In [28]:
predictor.delete_endpoint()

In [12]:
# !cp model/code/requirements_cpu.txt model/code/requirements.txt
# !cd model && tar -czvf ../model-inference-cpu.tar.gz *
# !aws s3 cp model-inference-cpu.tar.gz s3://sagemaker-cn-northwest-1-969422986683/uie/model-inference-cpu.tar.gz
    
import os
import sagemaker
import time
from sagemaker.pytorch.model import PyTorchModel

start = time.time()

sagemaker_session = sagemaker.Session()

role = sagemaker.get_execution_role()

instance_type = 'ml.m5.4xlarge'

model_data='s3://sagemaker-cn-northwest-1-969422986683/uie/model-inference-cpu.tar.gz'

pytorch_model = PyTorchModel(model_data=model_data, role=role,
                             entry_point='infer_cpu.py', framework_version='1.9.0', py_version='py38', model_server_workers=1)

predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

end = time.time()
print('time:', end-start) #10分钟左右

# # !pip install -i https://opentuna.cn/pypi/web/simple/ numpy==1.21.6
# import os
# import json

# import argparse
# from pprint import pprint

# import paddle
# import sys
# sys.path.append('./model/code') #folder which contains model, snn etc.,
# import time

# # from uie_predictor import UIEPredictor
# import numpy as np
# import paddle.nn.functional as F
# from paddlenlp.utils.log import logger
# from paddle.io import DataLoader, BatchSampler
# from paddlenlp.data import DataCollatorWithPadding
# from paddlenlp.datasets import load_dataset
# from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
# import re

# paddle.device.set_device("gpu")

# class NpEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         if isinstance(obj, np.floating):
#             return float(obj)
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return json.JSONEncoder.default(self, obj)

# def input_fn(request_body, request_content_type):
#     if request_content_type == 'application/json':
#         input_data = json.loads(request_body)
#         return input_data
#     else:
#         # Handle other content-types here or raise an Exception
#         # if the content type is not supported.  
#         return request_body
    

# def model_fn(model_dir):
# #     args = parse_args()
# #     args.model_path_prefix = os.path.join(model_dir, 'inference')
# # #     args.device = 'cpu'
# #     args.device = 'gpu'
# #     args.schema = ['债务人', '破产管理人']
# #     predictor1 = UIEPredictor(args)
    
#     params_path = model_dir       # 模型checkpoint文件夹的路径
#     model = AutoModelForSequenceClassification.from_pretrained(params_path)
#     model.eval()
#     tokenizer = AutoTokenizer.from_pretrained(params_path)
#     return (model,tokenizer)#predictor1,
  
# def predict_fn(input_data, model):
# #     print('predict_fn',model)
#     if input_data[0]=='sagemaker-kwm-iue': # model name needs to mapped
# #         print('use model sagemaker-kwm-iue')
# #         output = model[2].predict([input_data[1]])
#         print('wrong id')
#         pass

#     elif input_data[0]=='sagemaker-kwm-new':# model name needs to mapped
#         idx2label = [1, 0]
#         tokenizer=model[1]
#         new_model = model[0]
#         inputs = tokenizer([input_data[1]], max_len_seq=512)
#         input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
#         input_ids, token_type_ids = paddle.to_tensor(input_ids), paddle.to_tensor(token_type_ids)
#         logits = new_model(input_ids, token_type_ids)
#         probs = F.softmax(logits, axis=1)
#         idx = paddle.argmax(probs, axis=1).numpy()
#         idx = idx.tolist()
#         output = [idx2label[i] for i in idx]
#     return output

# def output_fn(prediction, content_type):
#     if content_type == 'application/json':
#         return json.dumps(prediction, ensure_ascii=False, cls=NpEncoder)
#     return prediction

# def parse_args():
#     parser = argparse.ArgumentParser()
# #     # Required parameters
# #     parser.add_argument(
# #         "--model_path_prefix",
# #         type=str,
# #         required=True,
# #         help="The path prefix of inference model to be used.", )
#     parser.add_argument(
#         "--position_prob",
#         default=0.5,
#         type=float,
#         help="Probability threshold for start/end index probabiliry.", )
#     parser.add_argument(
#         "--use_fp16",
#         action='store_true',
#         help="Whether to use fp16 inference, only takes effect when deploying on gpu.",
#     )
#     parser.add_argument(
#         "--max_seq_len",
#         default=512,
#         type=int,
#         help="The maximum input sequence length. Sequences longer than this will be split automatically.",
#     )
    
#     parsed, unknown = parser.parse_known_args() # this is an 'internal' method

#     for arg in unknown:
#         if arg.startswith(("-", "--")):
#             # you can pass any arguments to add_argument
#             parser.add_argument(arg, type=str)

#     args = parser.parse_args()
#     return args


# if __name__ == "__main__":
#     model = model_fn('/home/ec2-user/SageMaker/paddlenlp_sagemaker/model/')
# #     texts = ['sagemaker-kwm-iue','债务人: cksdafgsfgsjhkvgdkjsV 破产管理人:kjLFHk;fhs;dhsd;f']
#     texts = ['sagemaker-kwm-new','查封担保人李凤香的位于农安镇宝塔街西侧富苑小区3栋4单元202室，建筑面积78.14平方米']
#     start = time.time()
#     result = predict_fn(texts, model)
#     result_json = output_fn(result, content_type='application/json')
#     end = time.time()

#     print('result_json',result_json)
#     print('time:', end-start) 


outputs:  [{'法院': [{'text': '北京市海淀区人民法院', 'start': 1, 'end': 11, 'probability': 0.9251759648323059}], '原告': [{'text': '张三', 'start': 36, 'end': 38, 'probability': 0.9924564361572266, 'relations': {'委托代理人': [{'text': '李四', 'start': 45, 'end': 47, 'probability': 0.698268473148346}]}}], '被告': [{'text': 'B公司', 'start': 65, 'end': 68, 'probability': 0.804964005947113, 'relations': {'委托代理人': [{'text': '赵六', 'start': 91, 'end': 93, 'probability': 0.6848900318145752}]}}]}]
time: 3.469820737838745


In [6]:
# !tar -czvf ../backup.tar.gz *
# !aws s3 cp ../backup.tar.gz s3://sagemaker-cn-northwest-1-969422986683/uie/backup.tar.gz
texts = ['sagemaker-kwm-iue','债务人: cksdafgsfgsjhkvgdkjsV 破产管理人:kjLFHk;fhs;dhsd;f']

payload = json.dumps(texts)
payload

'["sagemaker-kwm-iue", "\\u503a\\u52a1\\u4eba: cksdafgsfgsjhkvgdkjsV \\u7834\\u4ea7\\u7ba1\\u7406\\u4eba:kjLFHk;fhs;dhsd;f"]'

[0]